In [36]:
#%pip install tpot

In [37]:
import pandas as pd
import umap.umap_ as umap
#import sweetviz #usado mais pra dados rotulados
import dtale
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import autokeras as ak
import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.utils import to_categorical

from sklearn import preprocessing
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold, cross_val_score,train_test_split 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_recall_fscore_support
from sklearn.neighbors import KNeighborsClassifier

import tpot
from tpot import TPOTClassifier

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

In [38]:
rootPath = "C:\\Users\\mlb\\Teste_VOCs\\1_Preprocessing\\NewDatasets"
columnsName_X = ["TGS826","TGS2611","TGS2603","TGS813","TGS822","TGS2602","TGS823","temperatura","pressao","umidade","culture_day"]

In [39]:
#Load albicans file data
#data dictionary
# Albicans = 1
# Glabrata = 2
# Haemulonii = 3
# Kodamaea_ohmeri = 4
# Krusei = 5
# Parapsilosis = 6

AllCandidasCollect    = pd.read_csv(rootPath+"\\AllCandidasCollect.csv", sep=",")

In [40]:
AllCandidasCollect

,TGS826,TGS2611,TGS2603,TGS813,TGS822,TGS2602,TGS823,temperatura,pressao,umidade,c_type,culture_day
0,91720.0,7305.0,6014.0,63460.0,4294.0,40700.0,8005.0,60.66,100.22,12.59,1,1
1,91910.0,7305.0,6020.0,63380.0,4293.0,40730.0,7998.0,60.71,100.15,12.52,1,1
2,91190.0,7305.0,6020.0,63460.0,4294.0,40750.0,7998.0,60.70,100.19,12.56,1,1
3,92380.0,7305.0,6020.0,64470.0,4319.0,40770.0,7998.0,60.69,83.83,12.60,1,1
4,35120.0,7305.0,7841.0,64730.0,2047.0,16630.0,7032.0,60.62,74.81,11.56,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
64961,87790.0,7171.0,8935.0,38260.0,2307.0,15180.0,7706.0,57.33,54.08,7.23,6,2
64962,86770.0,7171.0,8941.0,38200.0,2305.0,15180.0,7710.0,57.32,54.00,7.25,6,2
64963,87190.0,7171.0,8941.0,38260.0,2307.0,15180.0,7704.0,57.30,54.08,7.40,6,2
64964,87190.0,7171.0,8941.0,38330.0,2297.0,15170.0,7706.0,57.29,54.08,7.25,6,2


In [41]:
def getMetrics(y_true, y_pred):
    precision, recall, fbeta_score, support = precision_recall_fscore_support(y_true, y_pred, average='micro')
    f_measure = 2* ((precision*recall)/(precision+recall))
    return   precision, recall, (1-precision), f_measure

In [42]:
X = AllCandidasCollect.drop(columns=['c_type'])
y = AllCandidasCollect['c_type']
y_sorted , y_description = pd.factorize(AllCandidasCollect['c_type'], sort=False)
categorical_labels_y = to_categorical(y_sorted)

In [43]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=2)  

In [44]:
def evaluate_model(model, n_interaction):
    array_acc = []
    array_precision = []
    array_recall = []
    array_f1 = []
    for i in range(n_interaction):
        accuracy = np.mean(cross_val_score(model, X, y, cv=StratifiedKFold(n_splits=10, shuffle=True)))
        array_acc.append(accuracy)

        precision = np.mean(cross_val_score(model, X, y, cv=StratifiedKFold(n_splits=10, shuffle=True), scoring='precision_macro'))
        array_precision.append(precision)

        recall = np.mean(cross_val_score(model, X, y, cv=StratifiedKFold(n_splits=10, shuffle=True), scoring='recall_macro'))
        array_recall.append(recall)

        f1 = np.mean(cross_val_score(model, X, y, cv=StratifiedKFold(n_splits=10, shuffle=True), scoring='f1_macro'))
        array_f1.append(f1)

        print(f"Iteração:{i} - Acurracy:{accuracy}")
    create_chart(array_acc)
    return np.mean(accuracy), np.mean(precision),np.mean(recall),np.mean(f1), np.std(accuracy)
    #return array_acc

In [45]:
def create_chart(array_acc):
    fig, graf = plt.subplots(figsize=(20,10))

    graf.set_title('Resultado Análise com kfod = 10')
    graf.plot(list(range(len(array_acc))), array_acc,color='#FF0000', label='Acurácia', linewidth=2, alpha=0.5)
    graf.plot(list(range(len(array_acc))),[np.mean(array_acc)]*len(array_acc),color='#0000FF', label='Média', linewidth=2, alpha=0.5)
    graf.plot(list(range(len(array_acc))),[np.median(array_acc)]*len(array_acc),color='#00FF00', label='Mediana', linewidth=2, alpha=0.5)
    graf.legend()
    print(f"Média da acurácia para {len(array_acc)} repetições: {np.mean(array_acc)}")

In [46]:
#knn = KNeighborsClassifier(n_neighbors = 6, weights= 'uniform', algorithm='kd_tree',  n_jobs =2)
#print(evaluate_model(knn, 30))

In [47]:
#from sklearn.neural_network import MLPClassifier
#mlp = MLPClassifier(random_state=5, max_iter=500, alpha=1e-5, hidden_layer_sizes=(500,), activation='relu')
#print(evaluate_model(mlp, 30))

In [48]:
#svm = SVC(kernel='linear', gamma='auto')
#print(evaluate_model(svm, 30))

In [49]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid','linear']}
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto", decision_function_shape='ovo')
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto",decision_function_shape='ovo')
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto",decision_function_shape='ovo')
    elif ktype == 3:
        # Linear kernal
        print("Rodando SVC...")
        return SVC(kernel='linear', gamma="auto",decision_function_shape='ovo')

In [62]:
svm = getClassifier(3)
svm.fit(X_train,y_train).score(X_train,y_train)
y_pred = svm.predict(X_val)

predictions_score = accuracy_score(y_pred, y_val)
p, r , e , f = getMetrics(y_pred, y_val)

Rodando SVC...


In [63]:
#print(f"Resultado para a MLP: Score  train:{np.mean(treinScore)}")
print(f"Resultado para a svm: Score  test:{predictions_score}")
#print(f"Resultado para a MLP: Desvio train:{np.std(treinScore)}")
#print(f"Resultado para a MLP: Desvio test:{predictions_score}")

print(f"Resultado para a svm: precision test:{np.mean(p)}")
print(f"Resultado para a svm: recall    test:{np.mean(r)}")
print(f"Resultado para a svm: error     test:{np.mean(e)}")
print(f"Resultado para a svm: f_measure test:{np.mean(f)}")

Resultado para a svm: Score  test:0.8277666615360936
Resultado para a svm: precision test:0.8277666615360936
Resultado para a svm: recall    test:0.8277666615360936
Resultado para a svm: error     test:0.17223333846390643
Resultado para a svm: f_measure test:0.8277666615360936


In [ ]:

min_max_scaler = preprocessing.MinMaxScaler()
#standardScaler = preprocessing.StandardScaler()
y = df['TypeOfGlass']
x_scaled = min_max_scaler.fit_transform(AllCandidasCollect.drop['c_type'])
#df_scaled = standardScaler.fit_transform(df)
x_scaled = pd.DataFrame(x_scaled, columns=columsNamesX)

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
y_test = y_test.to_numpy()
y_train = y_train.to_numpy()
X_train = X_train.to_numpy()
X_test  = X_test.to_numpy()

In [ ]:
#clf = ak.StructuredDataClassifier(overwrite=True, max_trials=3)
# Feed the structured data classifier with training data.
#clf.fit(X_train, y_train, epochs=10)
# Predict with the best model.
#predicted_y = clf.predict(X_test)
# Evaluate the best model with testing data.
#print(clf.evaluate(X_test, y_test))

In [ ]:
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search
#model = TPOTClassifier(generations=5, population_size=50, cv=cv, scoring='accuracy', verbosity=2, random_state=1, n_jobs=-1)
# perform the search
#model.fit(X, y)
# export the best model
#model.export('tpot_best_model.py')

In [ ]:
input_dim = X_train.shape[1]
print(input_dim)
nb_classes = len(categorical_labels_y[0])
print(nb_classes)
print(np.shape(X_train))
print(np.shape(y_train))

In [ ]:
from tensorflow.keras.optimizers import SGD
num_epochs = 500
batch_size = 5000

array_history = []
array_score = []

for i in range(0,1):
    print("################## ITERAÇÃO {} #############################".format(i))
    model = Sequential()

    # pamarâmetros Dense 17 = número de nós de entrada (input_dim*2)-1,
    # input_dim = número de dimensões do problema 
    # relu = função de ativação
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,categorical_labels_y, test_size=0.3, random_state=2)
    
    model.add(Dense(((input_dim*4)-1), input_dim=input_dim, activation='relu')) # camada de entrada

    model.add(Dense(128,  activation="relu"))

    model.add(Dense(128,  activation="relu"))

    model.add(Dense(128,  activation="relu"))

    model.add(Dense(128,  activation="relu"))

    model.add(Dense(nb_classes, activation='softmax')) # camada de saída (o valor 6 é o número de classes do target)
    
    print(model.output_shape)
    
    model.summary()
    sgd = SGD(learning_rate=0.01, decay=1e-3, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    hiatorical_s = model.fit(X_train, y_train, epochs=num_epochs,  batch_size=batch_size)
    array_history.append(hiatorical_s)
    score_s = model.evaluate(X_test, y_test, batch_size=batch_size)
    array_score.append(score_s)
    print("Score: {}% - Acurácia: {}%".format(score_s[0]*100,score_s[1]*100 ))

In [ ]:
#Usa os dados reescalonados para a redução de dimensionalidade
embedding = reducer.fit_transform(scaled_all_candidas_df)
embedding.shape

In [ ]:
all_candidas_data2

In [ ]:
fig = plt.figure(figsize=[15,12])

ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(embedding[:, 0], embedding[:, 1], embedding[:, 2], c=all_candidas_data.c_type, cmap="tab10", s=30)
handles, labels = scatter.legend_elements()
legend2 = ax.legend(handles, ["C. Albicans", "C.Glabrata","C. Haemulonii","C. Kodamaea_ohmeri", "C. Krusei","C. Parapsilosis"], loc="upper right", title="Candidas types")

# make simple, bare axis lines through space:
xAxisLine = ((min(embedding[:, 0]), max(embedding[:, 0])), (0, 0), (0,0))
ax.plot(xAxisLine[0], xAxisLine[1], xAxisLine[2], 'r')
yAxisLine = ((0, 0), (min(embedding[:, 1]), max(embedding[:,1])), (0,0))
ax.plot(yAxisLine[0], yAxisLine[1], yAxisLine[2], 'r')
zAxisLine = ((0, 0), (0,0), (min(embedding[:, 2]), max(embedding[:, 2])))
ax.plot(zAxisLine[0], zAxisLine[1], zAxisLine[2], 'r')

# label the axes
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
ax.set_title("UMAP projection from All Candidas dataset")

In [ ]:
# d = dtale.show(embedding)
# d.open_browser()

In [ ]:
#Usa as classes do array original pra plotar a 
# Albicans = 1
# Glabrata = 2
# Haemulonii = 3
# Kodamaea_ohmeri = 4
# Krusei = 5
# Parapsilosis = 6
ax= sns.scatterplot(
    embedding[:, 0],
    embedding[:, 1],
    hue= all_candidas_data.c_type , s=15, cmap="tab10")
plt.gca().set_aspect('equal', 'datalim')
handles, labels  =  ax.get_legend_handles_labels()
#"C. Albicans", "C.Glabrata","C. Haemulonii","C. Albicans", "C.Glabrata","C. Haemulonii"
plt.legend(handles, ["C. Albicans", "C.Glabrata","C. Haemulonii","C. Kodamaea_ohmeri", "C. Krusei","C. Parapsilosis"], loc='lower right')
plt.title('UMAP projection of the All Candidas dataset', fontsize=24)
plt.show

In [ ]:
plt.scatter(embedding[:, 0], embedding[:, 1], c=all_candidas_data.c_type, cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(7)).set_ticks(np.arange(7))
plt.title('UMAP projection of the All Candidas dataset', fontsize=24);

In [ ]:
scaled_all_candidas_df

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(scaled_all_candidas_df)
result = pd.DataFrame(data = principalComponents
             , columns = ['PCA0', 'PCA1','PCA2'])

In [ ]:
fig = plt.figure(figsize=[15,12])

ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(result['PCA0'], result['PCA1'], result['PCA2'], c=all_candidas_data.c_type, cmap="tab10", s=30)
handles, labels = scatter.legend_elements()
legend2 = ax.legend(handles, ["C. Albicans", "C.Glabrata","C. Haemulonii","C. Kodamaea_ohmeri", "C. Krusei","C. Parapsilosis"], loc="upper right", title="Candidas types")

# make simple, bare axis lines through space:
xAxisLine = ((min(result['PCA0']), max(result['PCA0'])), (0, 0), (0,0))
ax.plot(xAxisLine[0], xAxisLine[1], xAxisLine[2], 'r')
yAxisLine = ((0, 0), (min(result['PCA1']), max(result['PCA1'])), (0,0))
ax.plot(yAxisLine[0], yAxisLine[1], yAxisLine[2], 'r')
zAxisLine = ((0, 0), (0,0), (min(result['PCA2']), max(result['PCA2'])))
ax.plot(zAxisLine[0], zAxisLine[1], zAxisLine[2], 'r')

# label the axes
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
ax.set_title("PCA projection from All Candidas dataset")

In [ ]:
components_df =  pd.DataFrame(abs(pca.components_[:3]),columns=scaled_all_candidas_df.columns,
                             index=['PCA1','PCA2','PCA3'])
components_df

#
# ['blue', 'orange', 'green', 'red', 'purple', 
#		'brown', 'pink', 'gray', 'olive', 'cyan']

In [ ]:
d = dtale.show(components_df)
d.open_browser()

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(scaled_all_candidas_df)
result = pd.DataFrame(data = principalComponents
             , columns = ['PCA0', 'PCA1'])

In [ ]:
ax = sns.scatterplot(data=result, x='PCA0', y='PCA1',cmap="tab10", s=50, c=all_candidas_data.c_type)
handles, labels  =  ax.get_legend_handles_labels()
handles, labels = scatter.legend_elements()
legend2 = ax.legend(handles, ["C. Albicans", "C.Glabrata","C. Haemulonii","C. Kodamaea_ohmeri", "C. Krusei","C. Parapsilosis"], loc="lower right", title="Candidas types")
plt.title('PCA projection from All Candidas dataset', fontsize=24)